# Семинар 10 - Нейросети

In [ ]:
!pip install torch torchvision

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
# warnings.simplefilter('ignore')
import torch
from torch.utils.data import DataLoader, Dataset, Subset
import torchvision
from torchvision import transforms
from matplotlib import pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch
print(torch.__version__)

## 1 Вспомним PyTorch

Нахождение сложной производной

Найдите производную по x от функции
$$\sin\left(\tan(x)\frac{x^2}{y} + \ln(e^{-x^2 + 3}+x^3y)\right)\tan(x^2e^{x^9})$$

При этом надо пользоваться встроенным в PyTorch autograd. Численное вычисление производной может не дать нужный результат.

In [ ]:
def find_x_derivative(x, y):
    # Ваш код здесь

    return

In [ ]:
find_x_derivative(1,21)

## 2 Подготовка данных

### В семинаре, будем использовать набор данных `fashion_mnist`, загрузим их

In [ ]:
from torchvision import datasets,transforms

In [ ]:
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train = True, download=True)

In [ ]:
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train = False, download = True)

In [ ]:
num_classes = len(trainset.classes)

In [ ]:
trainset.classes

In [ ]:
x_train = trainset.train_data
y_train = trainset.train_labels

x_test = testset.train_data
y_test = testset.train_labels

In [ ]:
fig = plt.figure(figsize=(15,5))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = features_idx[img_num]
    ax.set_title(trainset.classes[i])
    plt.imshow(im, cmap='gray_r')
plt.show()

### Проведем небольшие предобработки

In [ ]:
x_train_flat = x_train.flatten(start_dim=1).float()
x_test_flat = x_test.flatten(start_dim=1).float()
print(f'Была размерность: {x_train.shape}, стала: {x_train_flat.shape}')
print(f'Была размерность: {x_test.shape}, стала: {x_test_flat.shape}')

In [ ]:
D_out = num_classes
D_in = x_train_flat.shape[1]

## 3 Определим архитектуру нейросети в pyTorch

(вариантов определения архитектуры много, но эти 2 варианта наиболее популярные)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, 128),
    torch.nn.Sigmoid(),
    torch.nn.Linear(128, 10),
    torch.nn.Sigmoid(),
    torch.nn.Linear(10, D_out),
    torch.nn.Softmax(dim=1)
)

In [ ]:
import torch.nn.functional as F

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, 128)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(128, 10)
        self.act2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(10, D_out)
        self.act3 = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        x = self.act3(x)
        return x

model = Model()

In [ ]:
model

In [ ]:
# Forward pass: compute predicted y by passing x to the model. Module objects
# override the __call__ operator so you can call them like functions. When
# doing so you pass a Tensor of input data to the Module and it produces
# a Tensor of output data.
y_pred = model(x_train_flat)

In [ ]:
y_pred.shape

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# Compute and print loss. We pass Tensors containing the predicted and true
# values of y, and the loss function returns a Tensor containing the
# loss.
loss_old = loss_fn(y_pred, y_train)
loss_old

In [ ]:
acc_old = accuracy_score(y_train.numpy(), y_pred.argmax(dim=1))
acc_old

In [ ]:
# Zero the gradients before running the backward pass.
model.zero_grad()

In [ ]:
# Backward pass: compute gradient of the loss with respect to all the learnable
# parameters of the model. Internally, the parameters of each Module are stored
# in Tensors with requires_grad=True, so this call will compute gradients for
# all learnable parameters in the model.
loss_old.backward()

In [ ]:
learning_rate = 1e-2

In [ ]:
# Update the weights using gradient descent. Each parameter is a Tensor, so
# we can access its gradients like we did before.
with torch.no_grad():
  # Ваш код здесь


In [ ]:
y_pred = model(x_train_flat.float())
loss_new = loss_fn(y_pred, y_train)
step = loss_new.item()-loss_old.item()

acc_new = accuracy_score(y_train, y_pred.argmax(dim=1))

print(f'Лосс: {loss_old.item()} -> {loss_new.item()}. Step {step} ')
print(f'Accuracy: {acc_old} -> {acc_new}')

In [ ]:
def batch_train(model, loss_fn, learning_rate, x, y):
  # Ваш код здесь

    return(loss.item())

In [ ]:
def train(model, n_epochs, batch_size, learning_rate,  X, y, X_test, y_test):
    acc_train_all = []
    loss_train_all = []
    acc_test_all = []
    loss_test_all = []

    for epoch in range(n_epochs):

        permutation = torch.randperm(X.size()[0])

        for i in tqdm(range(0,X.float().size()[0], batch_size)):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X[indices], y[indices]
            batch_train(model, loss_fn, learning_rate, batch_x, batch_y)

        y_test_pred = model(X_test)
        y_train_pred = model(X)


        acc_train = accuracy_score(y, y_train_pred.argmax(dim=1))
        loss_train = loss_fn(y_train_pred, y).detach()
        acc_test = accuracy_score(y_test, y_test_pred.argmax(dim=1))
        loss_test = loss_fn(y_test_pred, y_test).detach()

        acc_train_all = np.append(acc_train_all, acc_train)
        loss_train_all = np.append(loss_train_all, loss_train)
        acc_test_all = np.append(acc_test_all, acc_test)
        loss_test_all = np.append(loss_test_all, loss_test)


        print(f'Epoch {epoch}: \n Accuracy - train: {acc_train} | test: {acc_test} \n Loss - train: {loss_train} | test: {loss_test}')

    return(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

In [ ]:
n_epochs = 10
batch_size = 1000
learning_rate = 1
model = Model()
acc_train_all, loss_train_all, acc_test_all, loss_test_all = \
          train(model, n_epochs, batch_size, learning_rate, x_train_flat, y_train, x_test_flat, y_test)


In [ ]:
def vis_history(acc_train_all, loss_train_all, acc_test_all, loss_test_all):
    fig = plt.figure(figsize=(16, 4))

    plt.subplot(1, 2, 1)

    plt.plot(loss_train_all, label='loss')
    plt.plot(loss_test_all, label='val_loss')

    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(acc_train_all, label='acc')
    plt.plot(acc_test_all, label='val_acc')

    plt.legend()
    plt.show()

In [ ]:
vis_history(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

## 4 Что мы можем улучшить?
- Отнормировать признаки
- Задать правила инициации весов
- Выбрать функцию активации
- Настроить скорость обучения

### 4.1 Нормировка
![picture](https://drive.google.com/uc?export=view&id=1kSw5ceu5iexG5DIPkhHGtXUepZtl9t6Q)

In [ ]:
import torch.nn.functional as F

class ModelNorm(torch.nn.Module):
    def __init__(self):
        super(ModelNorm, self).__init__()
        self.act1 = torch.nn.Sigmoid()
        self.fc1 = torch.nn.Linear(D_in, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.bn2 = torch.nn.BatchNorm1d(10)
        self.fc3 = torch.nn.Linear(10, D_out)
        self.act2 = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act1(x)
        x = self.fc3(x)
        x = self.act2(x)
        return x

model_norm = ModelNorm()

In [ ]:
mean = x_train_flat.mean()
std = x_train_flat.std()

x_train_norm = (x_train_flat - mean) / std
x_test_norm = (x_test_flat - mean) / std

In [ ]:
x_train_norm.max(), x_train_norm.min()

In [ ]:
n_epochs = 10
batch_size = 1000
learning_rate = 1
acc_train_all, loss_train_all, acc_test_all, loss_test_all = train(model_norm, n_epochs, batch_size, learning_rate,
                                                                   x_train_norm, y_train, x_test_norm, y_test)


In [ ]:
vis_history(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

### 4.2 Функции активации
![picture](https://drive.google.com/uc?export=view&id=1pHhpRQNXHnFCyVyvJJJtqMkLhJInT1i3)

In [ ]:
class ModelReLU(torch.nn.Module):
    def __init__(self):
        super(ModelReLU, self).__init__()
        self.act1 = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(D_in, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.bn2 = torch.nn.BatchNorm1d(10)
        self.fc3 = torch.nn.Linear(10, D_out)
        self.act2 = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act1(x)
        x = self.fc3(x)
        x = self.act2(x)
        return x

model_relu = ModelReLU()

In [ ]:
n_epochs = 10
batch_size = 1000
learning_rate = 1

acc_train_all, loss_train_all, acc_test_all, loss_test_all = train(model_relu, n_epochs, batch_size, learning_rate, x_train_norm, y_train, x_test_norm, y_test)


In [ ]:
vis_history(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

### 4.3 [Инициациия весов](https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/)
__Случайно__  
$ w = a * random$, но тогда если $a \gg 1$, то на выходе $b\gg1$ и если $a \ll 1 $, то $b \approx 0 $  

__Xavier__  
$a = \frac{1}{\sqrt{n}}$, где $n$ - кол-во нейронов на входе

__He__  
$a = \frac{1}{\sqrt{\frac{n}{2}}}$, где $n$ - кол-во нейронов на входе

In [ ]:
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.01)

Примените к модели  функцию инициации весов с помощью метода .apply()

In [ ]:
model_relu = ModelReLU()

In [ ]:
model_he = model_relu.apply(init_weights)

In [ ]:
n_epochs = 10
batch_size = 1000
learning_rate = 1

acc_train_all, loss_train_all, acc_test_all, loss_test_all = train(model_he, n_epochs, batch_size, learning_rate,
                                                                   x_train_norm, y_train, x_test_norm, y_test)


In [ ]:
vis_history(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

### 4.4 Влияние метода оптимизации градиентного спуска

![picture](https://drive.google.com/uc?export=view&id=1D3hA1C9MCx5eNRHb2yo2W_4wJjT0IjHv)

**Momentum**

Вместо того, чтобы использовать только градиент текущего шага, мы будем накапливать импульс градиента прошлых шагов для определения направления движения.
В связи со стохастической природой, обновления градиента происходят "зигзагообразно", с помощью момента мы усиливаем движение вдоль основного направления. На практике коэффициент у момента инициализируется на уровне 0,5 и постепенно увеличивается до 0,9 в течение нескольких эпох.
  
**RMSProp** (Root Mean Square Propogation)   

Мы обновляяем меньше веса, которые слишком часто обновляются, и будем использовать усреднённый по истории квадрат градиента.

**Adam** (Adaptive moment estimation)

Cочетает в себе и идею накопления движения и идею более слабого обновления весов для типичных признаков

#### Update batch train

In [ ]:
def batch_train(model, optimizer, x, y):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    model.zero_grad()
    loss.backward()
    #    with torch.no_grad():
    #    for param in model.parameters():
    #        param -= learning_rate * param.grad
    optimizer.step()
    return (loss.item())

#### Prepare models

In [ ]:
model_relu = ModelReLU()
model_he_SGD = model_relu.apply(init_weights)

In [ ]:
model_relu = ModelReLU()
model_he_Momentum = model_relu.apply(init_weights)

In [ ]:
model_relu = ModelReLU()
model_he_Adam = model_relu.apply(init_weights)

#### Prepare optimizers

In [ ]:
optimizerSGD = torch.optim.SGD(model_he_SGD.parameters(), lr=0.01, momentum=0.0)

In [ ]:
optimizerMomentum = torch.optim.SGD(model_he_Momentum.parameters(), lr=0.01, momentum=0.9)

In [ ]:
optimizerAdam = torch.optim.Adam(model_he_Adam.parameters(), lr=0.01)

#### Train

In [ ]:
n_epochs = 10
batch_size = 1000


loss_test_sgd = []
loss_test_sgd_moment = []
loss_test_adam = []

for epoch in range(n_epochs):

    permutation = torch.randperm(x_train_norm.size()[0])

    for i in tqdm(range(0,x_train_norm.float().size()[0], batch_size)):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_train_norm[indices], y_train[indices]
        batch_train(model_he_SGD, optimizerSGD, batch_x, batch_y)
        batch_train(model_he_Momentum, optimizerMomentum, batch_x, batch_y)
        batch_train(model_he_Adam, optimizerAdam, batch_x, batch_y)

    y_test_pred = model_he_SGD(x_test_norm)
    loss_train = loss_fn(y_test_pred, y_test).detach().numpy()
    print(f' SGD Epoch: {epoch} loss {loss_train}')
    loss_test_sgd.append(loss_train)

    y_test_pred = model_he_Momentum(x_test_norm)
    loss_train = loss_fn(y_test_pred, y_test).detach().numpy()
    print(f' Momentum Epoch: {epoch} loss {loss_train}')
    loss_test_sgd_moment.append(loss_train)

    y_test_pred = model_he_Adam(x_test_norm)
    loss_train = loss_fn(y_test_pred, y_test).detach().numpy()
    print(f' Adam Epoch: {epoch} loss {loss_train}')
    loss_test_adam.append(loss_train)

In [ ]:
fig = plt.figure(figsize=(16, 4))

plt.plot(loss_test_sgd, label='SGD')
plt.plot(loss_test_sgd_moment, label='SGD with momentum')
plt.plot(loss_test_adam, label='Adam')

plt.legend()
plt.show()

### 4.5 Влияние скорости обучения
Посмотрим, как влияет параметр `learning_rate` на качество нашей модели на обучающей выборке

In [ ]:
model_relu = ModelReLU()
model_he_Adam = model_relu.apply(init_weights)

In [ ]:
optimizerAdam = torch.optim.Adam(model_he_Adam.parameters(), lr=0.01)

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

scheduler = lr_scheduler.LinearLR(optimizerAdam, start_factor=1.0, end_factor=0.01, total_iters=10)

In [ ]:
def train(model, optimizer, scheduler, n_epochs, batch_size,  X, y, X_test, y_test):
    acc_train_all = []
    loss_train_all = []
    acc_test_all = []
    loss_test_all = []

    for epoch in range(n_epochs):

        permutation = torch.randperm(X.size()[0])

        for i in tqdm(range(0,X.float().size()[0], batch_size)):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X[indices], y[indices]
            batch_train(model, optimizer, batch_x, batch_y)

        scheduler.step()

        y_test_pred = model(X_test)
        y_train_pred = model(X)


        acc_train = accuracy_score(y, y_train_pred.argmax(dim=1))
        loss_train = loss_fn(y_train_pred, y).detach()
        acc_test = accuracy_score(y_test, y_test_pred.argmax(dim=1))
        loss_test = loss_fn(y_test_pred, y_test).detach()

        acc_train_all = np.append(acc_train_all, acc_train)
        loss_train_all = np.append(loss_train_all, loss_train)
        acc_test_all = np.append(acc_test_all, acc_test)
        loss_test_all = np.append(loss_test_all, loss_test)


        print(f'Epoch {epoch}: \n Accuracy - train: {acc_train} | test: {acc_test} \n Loss - train: {loss_train} | test: {loss_test}')

    return(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

In [ ]:
n_epochs = 10
batch_size = 1000

acc_train_all, loss_train_all, acc_test_all, loss_test_all = train(model_he_Adam, optimizerAdam, scheduler, n_epochs, batch_size,
                                                                    x_train_norm, y_train, x_test_norm, y_test)

In [ ]:
vis_history(acc_train_all, loss_train_all, acc_test_all, loss_test_all)

# Ссылки
- Материалы курса [dlschool](https://www.dlschool.org/)
- [Курс "Deep learning на пальцах", лекция 4](https://youtu.be/tnrbx7V9RbA)
- [Статья: Оптимизация градиентного спуска](http://ruder.io/optimizing-gradient-descent/)
- [Статья: Методы оптимизации нейронных сетей](https://habr.com/ru/post/318970/)
- YSDA [Practical RL course week04 materials](https://github.com/yandexdataschool/Practical_RL/tree/master/week04_%5Brecap%5D_deep_learning).
- PyTorch official tutorials and [this kaggle kernel](https://www.kaggle.com/pinocookie/pytorch-dataset-and-dataloader)
- PyTorch tutorial by [Stanford CS 231n course](http://cs231n.stanford.edu)